In [21]:
#Importing the libraries
import pandas as pd 
import numpy as np
from typing import List
import langchain
from langchain_core.documents import Document
from pinecone import Pinecone
from pinecone import ServerlessSpec
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAI
from langchain.chat_models.base import init_chat_model
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [8]:
#Function to load each excel file as a separate document
def load_Excel(file_path:str)-> List[Document]:
    document=[]
    excelfile=pd.ExcelFile(file_path)
    for sheet_name in excelfile.sheet_names:
        df=pd.read_excel(file_path,sheet_name=sheet_name)

        sheet_content=f"Sheet Name: {sheet_name} \n"
        sheet_content+=f"Column Name: {' ,'.join(df.columns)} \n"
        sheet_content+=f"Number of Records: {len(df)} \n"
        sheet_content+=df.to_string(index=False)
        doc=Document(page_content=sheet_content,metadata={"Data Source":"Excel File","Source":file_path,"Number of Records":len(df)
                                                          ,"Number of Columns":len(df.columns),"Month":sheet_name
                                                          })
        document.append(doc)
    return document
        
    




In [51]:
#Loading the excel file to create list of documents
documents=load_Excel("/Users/aswinganapathysubramanian/Documents/PersonalProjects/Test/Finance.xlsx")

In [52]:
# Initialize the OpenAI and Pinecone api keys in the environment (Note add the api keys in the .env file)
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["PINECONE_API_KEY"]=os.getenv("PINECONE_API_KEY")

In [53]:
# Initializing Embedding, we are using openai embedding
embeddings=OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)

In [54]:
#Split the documents into chunks
#Initializing the text splitter
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    separators=[" ","\n"]
)

chunks=text_splitter.split_documents(documents)

In [55]:
#Initializing  the instance for the Vector Database
pinecone_api_key=os.getenv("PINECONE_API_KEY")
pc=Pinecone(api_key=pinecone_api_key)

In [56]:
# Checking if the index is already present
index_name="personalfinance"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws",region="us-east-1")
    )

In [57]:
#Accessing the index
index=pc.Index(index_name)

In [58]:
# Initializing the Vector Store
vector_store=PineconeVectorStore(index=index,embedding=embeddings)

In [59]:
# Adding the chunks to the vector store
vector_store.add_documents(documents)

['bffeb65b-4fbd-4edf-b0fd-0842924e486d',
 '7e4a4f36-16ca-4489-bc21-584fd6afada1',
 '6173602d-c6d1-422c-b0ba-18e53a5b177e',
 'd3443686-dff3-46d5-97a9-4905b794187c',
 '827e74d0-ebdc-4113-a193-2e25c899c2dc',
 '12ff00ba-c8c5-4de4-b8f6-617b215533a1',
 '83e85d57-2d64-4d27-bbf0-6727149a99f0',
 '4c74e152-9037-4c99-b945-9e1270b44273',
 '6beb556b-6321-487e-b04f-f1bfe4ebbeef',
 '3908fc65-8db5-482c-a1e7-4746067200e9',
 '65010404-4824-4448-bf92-c1eb86d088c2',
 '55c3ea14-a73d-40e2-8898-e3ad64780b4f']

In [40]:
# initializing the vector store as retriever
retriever= vector_store.as_retriever(search_kwargs={"k":3})

In [41]:
retriever

VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x120da1450>, search_kwargs={'k': 3})

In [42]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-5.2",temperature=0.2, max_completion_tokens=500)

In [43]:
## create a prompt that includes the chat history
contextualize_q_system_prompt = """Given a chat history and the latest user question 
which might reference context in the chat history, formulate a standalone question 
which can be understood without the chat history. Do NOT answer the question, 
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt= ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human","{input}"),
])

In [44]:
# Create a history aware retriever
history_aware_retriever=create_history_aware_retriever(llm,retriever, contextualize_q_prompt)

In [45]:
#Create a new document chain with history
qa_system_prompt="""You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

Context: {context}"""

qa_prompt=ChatPromptTemplate.from_messages([
    ("system",qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human","{input}")
])

question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)

In [46]:
Conversation_Rag_chain=create_retrieval_chain(history_aware_retriever,
                                              question_answer_chain)

In [47]:
#Invoking conversational Retriever
chat_history=[]
result1= Conversation_Rag_chain.invoke({
    "chat_history":chat_history,
    "input": "What is my major spending for January"
})

print(f"Q: What is my major spending for January")
print(f"A: {result1['answer']}")

Q: What is my major spending for January
A: In January, the largest single expense is **Furniture (Ikea)** for **$686.13** (Harini, 2025-01-25). The next biggest major spend is **Car EMI**: **$590.00** (Aswin, 2025-01-16) and **$522.52** (Harini, 2025-01-16).


In [48]:
chat_history.extend([
    HumanMessage(content="What is my major spending for January"),
    AIMessage(content=result1['answer'])
])

In [50]:
##Follow Up question
result2= Conversation_Rag_chain.invoke({
    "chat_history":chat_history,
    "input": "WHich person spent the most in January based on data provided?" #Refers to ML from Previous Question
})

print(f"Q: WHo spent the most in January?")
print(f"A: {result2['answer']}")

Q: WHo spent the most in January?
A: **Harini** spent the most in January: **$1,997.24** total vs **Aswin** at **$1,634.21**.
